In [5]:
!curl -o input.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1089k  100 1089k    0     0  4104k      0 --:--:-- --:--:-- --:--:-- 4141k


In [8]:
with open('input.txt', 'r') as f:
    text = f.read()

print(f"Length of dataset in characters: {len(text)}")

Length of dataset in characters: 1115394


In [9]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
text_vocab = sorted(list(set(text)))

print("".join(text_vocab))

print(f"Length of vocabulary: {len(text_vocab)}")


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
Length of vocabulary: 65


In [26]:
# character to integer mapping. 
stoi = {ch:i for i, ch in enumerate(text_vocab)}
itos = {i:ch for i, ch in enumerate(text_vocab)}

# tiktoken(openai byte-pair-encoding)/sentencepiece(google) are usually used for this in large scale models

# take a string and output a list of integers
encode = lambda s: [stoi[c] for c in s]

# take a list and output a string
decode = lambda l: ''.join([itos[i] for i in l])

encoded_string = encode("Hello brother")
print(encoded_string)
print(decode(encoded_string))

[20, 43, 50, 50, 53, 1, 40, 56, 53, 58, 46, 43, 56]
Hello brother


In [32]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)

In [36]:
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [ ]:
# separate the dataset into a train and validation split

n = int(0.9*len(data)) # first 90% will be train, rest val

train_data = data[:n] 
val_data = data[n:]

In [39]:
block_size = 8 # context length: how many characters do we look at when predicting the next one?
train_data[:block_size+1] # example of input(8) and target(9th character) for the first 9 characters in the dataset. We will use this to train the model.

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [40]:
x = train_data[:block_size] # first 8 characters
y = train_data[1:block_size+1] # next 8 characters, which are

for t in range(block_size):
    context = x[:t+1] # the context we have available at time t
    target = y[t] # the next character we want to predict
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [ ]:
torch